# TikTok data and LLM coding

### Setup 

Run the following cells to set up this notebook

In [3]:
import ast
from IPython.display import display, clear_output
import json
from ipywidgets import widgets, VBox, HBox, Button, Label, Output
import krippendorff
import pandas as pd
import pyktok as pyk
import random
from openai import OpenAI
import seaborn as sns
import matplotlib.pyplot as plt

from dotenv import load_dotenv

_ = load_dotenv()

If pyktok does not operate as expected, you may find it helpful to run the 'specify_browser' function. 'specify_browser' takes as its sole argument a string representing a browser installed on your system, e.g. "chrome," "firefox," "edge," etc.


In [4]:
client = OpenAI()

In [5]:
import nest_asyncio
nest_asyncio.apply()

## Exploring Public Opinion in TikTok Comments

This week we will be exploring TikTok's role in shaping public opinion. Social media has become a popular site of discourse, and platforms like TikTok have been credited for shaping popular culture, politics, and everyday interactions. TikTok comment sections are a site where public opinion is debated and negotiated. At the same time, platform-level decisions alter what comments are most visible on comment pages.

## Choosing a Debate

Create a group of 2 (where at least one person in the group has a TikTok account) and find a video related to popular culture that has discusses two clearly defined positions as a part of a greater debate. This video should also have at least 10k comments. Copy the url of the video and paste it below.

In [6]:
# Replace with your video URL
video_url = "https://www.tiktok.com/@jimmyonrelationships/video/7286450127701708078?_r=1&_t=ZT-91Gew7Z8evv"

## Scrape Comments!

In [7]:
async def get_comments():
    pyk.save_tiktok_comments(video_url,
                         comment_count=1000,
                         save_comments=True,
                         return_comments=False,)

await get_comments()

Error: BrowserType.launch: Executable doesn't exist at /Commjhub/jupyterhub/home/praachic/.cache/ms-playwright/chromium-1187/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝

## Define a Research Question
What are you hoping to learn by studying comments on this video?

Is the conflict stronger rooted in an excess of mental load (as the wife feels) or is it a greater issue of commuication of needs (as the husband feels)? Do the majority of the comments agree with the wife or husband? Alternatively, do they find both partners equally guilty for a lack of communication?

## Defining Concepts
Quantitative content analysis is a popular method for translating non-numeric content into numeric, quantitative data. To do so, researchers create clearly defined and validated concepts. The first step in studying our TikTok debate is clearly defining how each position should be coded.

In [16]:
concept1 = "Code 1 if the comment agrees with the wife's point of view regarding exhaustion, mental load, and long-term unsustainability (This includes comments that find both parties equally convincing and both in the wrong). Code 0 if the comment does not agree with the wife or if the comment is unrelated to the situation in the video." 

concept2 = "Code 1 if the comment agrees with the husband's point of view regarding a lack of communication of wants and unreasonable expectations. (This includes comments that find both parties equally convincing and both in the wrong). Code 0 if the comment does not agree with the husband or if the comment is unrelated to the situation in the video."

## Let's Code!

In [12]:
# !!Replace with your filename!!
df = pd.read_csv('@jimmyonrelationships_video_7286450127701708078_comments.csv') 

sample_df = df.sample(n=100, random_state=42).reset_index(drop=True)
text_col = 'text'

for col in ['coder1_concept1', 'coder1_concept2', 'coder2_concept1', 'coder2_concept2']:
    sample_df[col] = None

main_out = Output()
coder1_out = Output()
coder2_out = Output()

def code_entries(coder_label, concept1_col, concept2_col, container_out, next_callback=None):
    index = 0
    text_label = Label(value=sample_df.loc[index, text_col])

    # !!Change the descriptions to your concepts!!
    concept1 = widgets.ToggleButtons(options=[0, 1], description='Concept 1:')
    concept2 = widgets.ToggleButtons(options=[0, 1], description='Concept 2:')
    
    next_button = Button(description='Next →', button_style='success')
    progress = Label(value=f"{coder_label}: {index + 1}/{len(sample_df)}")

    def next_entry(_):
        nonlocal index
        sample_df.loc[index, concept1_col] = concept1.value
        sample_df.loc[index, concept2_col] = concept2.value

        index += 1
        if index < len(sample_df):
            text_label.value = sample_df.loc[index, text_col]
            concept1.value = None
            concept2.value = None
            progress.value = f"{coder_label}: {index + 1}/{len(sample_df)}"
        else:
            container_out.clear_output()
            with container_out:
                display(Label(value=f"{coder_label} has finished coding! ✅"))
                if next_callback:
                    next_callback()
                else:
                    with main_out:
                        print("✅ Both coders finished. Saving data...")
                        sample_df.to_csv('coded_data.csv', index=False)
                        print("💾 Coded data saved to coded_data.csv")

    next_button.on_click(next_entry)

    with container_out:
        container_out.clear_output()
        display(VBox([progress, text_label, concept1, concept2, next_button]))

# ---- Workflow ----

def start_coder2():
    print("\n=== Coder 2: Please start coding ===")
    code_entries("Coder 2", "coder2_concept1", "coder2_concept2", coder2_out)

print(f"Concept 1: {concept1}")
print(f"Concept 2: {concept2}\n")

print("=== Coder 1: Please start coding ===")
code_entries("Coder 1", "coder1_concept1", "coder1_concept2", coder1_out, next_callback=start_coder2)

display(coder1_out, coder2_out, main_out)


Concept 1: Code 1 if the comment agrees with the wife's point of view regarding exhaustion, mental load, and long-term unsustainability (This includes comments that find both parties equally convincing and both in the wrong). Code 0 if the comment does not agree with the wife or if the comment is unrelated to the situation in the video.
Concept 2: Code 1 if the comment agrees with the husband's point of view regarding a lack of communication of wants and unreasonable expectations. (This includes comments that find both parties equally convincing and both in the wrong). Code 0 if the comment does not agree with the husband or if the comment is unrelated to the situation in the video.

=== Coder 1: Please start coding ===


Output()

Output()

Output()

## Assess Interrater Reliability

In [8]:
df = pd.read_csv("coded_data.csv")

def krippendorff_alpha_for_concept(df, concept_name):
    data = [
        df[f'coder1_{concept_name}'].tolist(),
        df[f'coder2_{concept_name}'].tolist()
    ]
    return krippendorff.alpha(reliability_data=data, level_of_measurement='nominal')

alpha_concept1 = krippendorff_alpha_for_concept(df, "concept1")
alpha_concept2 = krippendorff_alpha_for_concept(df, "concept2")

print("Krippendorff’s Alpha Scores:")
print(f"• Concept 1: {alpha_concept1:.3f}")
print(f"• Concept 2: {alpha_concept2:.3f}")

Krippendorff’s Alpha Scores:
• Concept 1: 0.620
• Concept 2: 0.468


## Evaluate the interrater reliability and iterate!

![](krippendorff.png)

Describe your interrater reliability. If you scored high, great! How did your concept descriptions help you code effectively? If you scored poorly, what might you have done better?

**Regardless of your score, discuss potential edge cases and improve your concept descriptions**

## Hire a LLM coder!
Write a prompt for an LLM to code the same 100 comments and calculate intercoder reliability. Be sure to tell the LLM their role!

In [25]:
LLM_prompt = '''You are a research assistant. 
                You will receive a series of tiktok comments, 
                please code for if the comment supports the wife or the husband.
                Is the conflict stronger rooted in an excess of mental load (as the wife feels) or is it a greater issue of commuication of needs (as the husband feels)? Do the majority of the comments agree with the wife or husband? Alternatively, do they find both partners equally guilty for a lack of communication?
                for human_concept1 = "Code 1 if the comment agrees with the wife's point of view regarding exhaustion, mental load, and long-term unsustainability (This includes comments that find both parties equally convincing and both in the wrong). Code 0 if the comment does not agree with the wife or if the comment is unrelated to the situation in the video." 
                for concept2 = "Code 1 if the comment agrees with the husband's point of view regarding a lack of communication of wants and unreasonable expectations. (This includes comments that find both parties equally convincing and both in the wrong). Code 0 if the comment does not agree with the husband or if the comment is unrelated to the situation in the video."
             
             '''

## Code with your LLM coder!
For the human coding, agree on a code with your partner for each of the concept

In [26]:
text_col = "text"
sample_df = df.sample(n=100,random_state = 11).reset_index(drop=True)
# sample_df["human_concept1"] = None
# sample_df["human_concept2"] = None

# def code_entries():
#     index = 0
#     out = Output()

#     text_label = widgets.Textarea(value=sample_df.loc[index, text_col], layout=widgets.Layout(width='600px', height='120px'))
#     concept1 = widgets.ToggleButtons(options=[0, 1], description="Concept 1:")
#     concept2 = widgets.ToggleButtons(options=[0, 1], description="Concept 2:")
#     next_button = Button(description="Next →", button_style="success")
#     progress = Label(value=f"Human Coding: {index + 1}/{len(sample_df)}")

#     def next_entry(_):
#         nonlocal index
#         sample_df.loc[index, "human_concept1"] = concept1.value
#         sample_df.loc[index, "human_concept2"] = concept2.value

#         index += 1
#         if index < len(sample_df):
#             text_label.value = sample_df.loc[index, text_col]
#             concept1.value = None
#             concept2.value = None
#             progress.value = f"Human Coding: {index + 1}/{len(sample_df)}"
#         else:
#             clear_output()
#             display(Label(value="✅ Human coding complete! Now AI will code the same sample."))
#             out.clear_output()

#     next_button.on_click(next_entry)
#     display(VBox([progress, text_label, concept1, concept2, next_button, out]))

# code_entries()

In [13]:
# sample_df.drop_duplicates(subset='cid').to_csv('human_coded_data2.csv')

In [27]:
batch_size = 10

def ai_code_batch(text_list):
    texts_formatted = "\n".join([f"{i+1}. {t}" for i, t in enumerate(text_list)])
    
    prompt = f"""
        {LLM_prompt}
        
        You will receive {len(text_list)} texts.
        
        Return ONLY valid JSON — a list of {len(text_list)} dictionaries like this:
        
        [
          {{"concept1": 1, "concept2": 0}},
          ...
        ]
        
        No explanations, no comments, no extra text — only the JSON array.
        
        Texts:
        {texts_formatted}
        """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    try:
        output_text = response.choices[0].message.content.strip()
        batch_json = json.loads(output_text)
        return batch_json  
    except Exception as e:
        try:
            print("Trying to parse")
            s = string.replace("```", "").replace("json", "").replace("\n", "").strip()
            lst = json.loads(s)
            print("✅ Parsed successfully.")
            return lst
        except Exception as e:
            print("⚠️ Parsing error:", e)

            return [{"concept1": None, "concept2": None} for _ in text_list]

In [28]:
sample_ai_concept1, sample_ai_concept2 = [], []

for start in range(0, len(sample_df), batch_size):
    print(f"Beginning coding round {batch_size/10}")
    end = start + batch_size
    sample_batch_texts = sample_df[text_col].iloc[start:end].tolist()
    sample_batch_results = ai_code_batch(sample_batch_texts)

    for r in sample_batch_results:
        sample_ai_concept1.append(r.get("concept1"))
        sample_ai_concept2.append(r.get("concept2"))

sample_df["ai_concept1"] = sample_ai_concept1
sample_df["ai_concept2"] = sample_ai_concept2

sample_df.to_csv("ai_coded_data.csv", index=False)
print("✅ Combined human + AI coded data saved to ai_coded_data.csv")


Beginning coding round 1.0
Beginning coding round 1.0
Trying to parse
⚠️ Parsing error: name 'string' is not defined
Beginning coding round 1.0
Beginning coding round 1.0
Trying to parse
⚠️ Parsing error: name 'string' is not defined
Beginning coding round 1.0
Trying to parse
⚠️ Parsing error: name 'string' is not defined
Beginning coding round 1.0
Beginning coding round 1.0
Trying to parse
⚠️ Parsing error: name 'string' is not defined
Beginning coding round 1.0
Beginning coding round 1.0
Trying to parse
⚠️ Parsing error: name 'string' is not defined
Beginning coding round 1.0
✅ Combined human + AI coded data saved to ai_coded_data.csv


## Evaluate your agreement
Repeat the process until you reach at least 0.7 agreeability with the LLM

In [31]:
sample_df

,allow_download_photo,author_pin,aweme_id,cid,collect_stat,comment_language,comment_post_item_ids,create_time,digg_count,image_list,...,trans_btn_style,user,user_buried,user_digged,coder1_concept1,coder1_concept2,coder2_concept1,coder2_concept2,ai_concept1,ai_concept2
0,False,False,7286450127701708078,7306252316528362282,0,en,NaN,1701119454,0,NaN,...,0,"{'account_labels': None, 'ad_cover_url': None,...",False,0,0,0.0,1,0.0,1.0,0.0
1,False,False,7286450127701708078,7298008523689788202,0,en,NaN,1699200043,3,NaN,...,0,"{'account_labels': None, 'ad_cover_url': None,...",False,0,1,0.0,0,1.0,1.0,0.0
2,True,False,7286450127701708078,7286889709934166814,0,en,NaN,1696611283,3,NaN,...,0,"{'account_labels': None, 'ad_cover_url': None,...",False,0,0,0.0,0,0.0,1.0,0.0
3,True,False,7286450127701708078,7289530032451273515,0,en,NaN,1697225999,6,NaN,...,0,"{'account_labels': None, 'ad_cover_url': None,...",False,0,1,0.0,1,0.0,1.0,0.0
4,True,False,7286450127701708078,7286631002574045953,0,en,NaN,1696551027,3,NaN,...,0,"{'account_labels': None, 'ad_cover_url': None,...",False,0,0,1.0,0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,False,7286450127701708078,7286541295496938282,0,en,NaN,1696530147,3,NaN,...,0,"{'account_labels': None, 'ad_cover_url': None,...",False,0,1,1.0,0,1.0,0.0,1.0
96,True,False,7286450127701708078,7454202661744640814,0,en,NaN,1735566835,0,NaN,...,0,"{'account_labels': None, 'ad_cover_url': None,...",False,0,1,0.0,1,0.0,0.0,0.0
97,True,False,7286450127701708078,7304075191033266976,0,en,NaN,1700612619,0,NaN,...,0,"{'account_labels': None, 'ad_cover_url': None,...",False,0,1,0.0,1,0.0,1.0,0.0
98,True,False,7286450127701708078,7312694165780087557,0,es,NaN,1702619322,51,NaN,...,1,"{'account_labels': None, 'ad_cover_url': None,...",False,0,0,0.0,0,0.0,0.0,0.0


In [32]:
def krippendorff_alpha_for_concept(df, concept_name):
    df.dropna(subset='cid', inplace=True)
    data = [
        df[f"coder1_{concept_name}"].tolist(),
        df[f"ai_{concept_name}"].tolist()
    ]
    return krippendorff.alpha(reliability_data=data, level_of_measurement='nominal')

alpha_c1 = krippendorff_alpha_for_concept(sample_df, "concept1")
alpha_c2 = krippendorff_alpha_for_concept(sample_df, "concept2")

print("Krippendorff’s Alpha between Human and AI:")
print(f"• Concept 1: {alpha_c1:.3f}")
print(f"• Concept 2: {alpha_c2:.3f}")


Krippendorff’s Alpha between Human and AI:
• Concept 1: 0.480
• Concept 2: 0.536


## Use your prompt to code the entire comments dataset

In [ ]:
# !! Change file name !!
comments = pd.read_csv("@abcnews_video_7413187084074159391_comments.csv")
comments = comments.head(100)

ai_concept1, ai_concept2 = [], []

for start in range(0, len(comments), batch_size):
    print(f"Beginning coding round {(start/10) + 1}")
    end = start + batch_size
    batch_texts = comments[text_col].iloc[start:end].tolist()
    batch_results = ai_code_batch(batch_texts)

    for r in batch_results:
        ai_concept1.append(r.get("concept1"))
        ai_concept2.append(r.get("concept2"))

# Add results to dataframe
comments["ai_concept1"] = ai_concept1
comments["ai_concept2"] = ai_concept2

# Save to file
comments.to_csv("final_coded_data.csv", index=False)
print("✅ Finished Coding!")

## Analyze your data

In [ ]:
final_coded_data = pd.read_csv("final_coded_data.csv")

#### Exploring quantity of comments

In [ ]:
counts = {
    "ai_concept1": final_coded_data["ai_concept1"].sum(),
    "ai_concept2": final_coded_data["ai_concept2"].sum()
}

count_df = pd.DataFrame(list(counts.items()), columns=["Concept", "Total_1s"])

plt.figure(figsize=(6, 4))
plt.bar(count_df["Concept"], count_df["Total_1s"])
plt.title("Total Count of 1s by AI Concept")
plt.ylabel("Count of 1s")
plt.xlabel("Concept")
plt.tight_layout()
plt.show()

#### What comments received more likes?

In [ ]:
def collapse(x):
    if x.ai_concept1 == 1:
        return "concept1"
    elif x.ai_concept2 == 1:
        return "concept2"
    else:
        return "neither"

final_coded_data['collapsed_codes'] = final_coded_data.apply(collapse, axis=1)

In [ ]:
sns.barplot(data=final_coded_data, y='collapsed_codes', x='digg_count')

#### What comments got the most replies?

In [ ]:
sns.barplot(data=final_coded_data, y='collapsed_codes', x='reply_comment_total')

#### What types of comments are shown more?
This is a visibility metric: higher scores means higher visibility in the comment section

In [ ]:
final_coded_data['sort_extra_score_loaded'] = final_coded_data.sort_extra_score.apply(lambda x: ast.literal_eval(x))
final_coded_data['show_more'] = final_coded_data.sort_extra_score_loaded.apply(lambda x: float(dict(x)['show_more_score']))
sns.barplot(data=final_coded_data, y='collapsed_codes', x='show_more')

#### Are there differences in age between commenters?

In [ ]:
final_coded_data['users_loaded'] = final_coded_data.user.apply(lambda x: ast.literal_eval(x))
final_coded_data['age'] = final_coded_data.users_loaded.apply(
    lambda x: x['predicted_age_group'] if isinstance(x, dict) and 'predicted_age_group' in x else None
)

def reduce(x):
    try:
        return float(x)
    except:
        return float('nan')
final_coded_data['age'] = final_coded_data.age.apply(reduce)


In [ ]:
sns.barplot(data=final_coded_data.dropna(subset='age'), y='collapsed_codes', x='age')

- group -1 = 13-17
- group 1 = 18-24
- group 2 = 25-34
- group 3 = 35-44
- group 4 = 45-54
- group 5 = 55+

#### Use an LLM to create one more visualization or analysis
You might need to inspect your data!

## What does it all mean??
Spend 2-3 paragraphs reflecting on the results from your analysis. What are the implications of this work?